## Situacion y problema a simular
La situacion que se busca simular es la limpieza del suelo de una habitación por medio de aspiradoras roboticas inteligentes.


### Consideraciones del modelo
El modelo tiene las siguientes consideraciones:
- Las celdas sucias deben generarse de manera aleatoria dentro de los
limites del espacio.
- Los agentes robot deben ser del tamaño del ancho del espacio y deben generarse
inicialmente en la primera fila.
- Se debe establecer el porcentaje inicial de celdas sucias totales con respecto
al tamaño total del espacio.
- El modelo debe ejecutarse sobre un tiempo maximo limitado.


## Agentes
En nuestro modelo hay presente dos tipo de agentes, los cuales son:
- DirtAgent
- RobotAgent

### Colores de los agentes
- RobotAgent -> Color negro
- DirtAgent -> Color cafe
NOTA: Las demas celdas desocupadas estan de color blanco.

### Acciones y percepciones
DirtAgent:
- Existir para indicarle al modelo las celdas sucias.
RobotAgent:
- Remover las celdas sucias.
- Moverse una unidad alrededor de sus 8 posibles posiciones.


## Interacción y comunicación entre agentes
###Interaccion entre un 'DirtAgent' y 'RobotAgent'
El agente robot es capaz de percibir las celdas que estan limpias y las que no. Esto con el proposito de ser capaz de remover a un agente suciedad y cumplir con su acción de limpieza. Por esto la  interaccion entre ambos consiste en que el agente robot remueve al agente suciedad del modelo.
###Interaccion entre un 'RobotAgent' y otro 'RobotAgent'
El agente robot es capaz de percibir a los demas agentes robots en su entorno. Esto con el proposito de determinar las posiciones a las que puede moverse y asi evitar una colision o sobreposicion entre agentes.


## El entorno
El modelo es una habitacion de MxN dimensiones la cual sera el lugar de interaccion entre los agentes Para este caso se decidio realizar un modelo para simular el comportamiento de aspiradoras robots que se mueven de forma aleatoria alrededor de la habitacion y tienen la capacidad de remover la suciedad del piso.


## Variables y parámetros
- Ancho de la habitación
- Largo de la habitación.
- El numero de agentes robots es el mismo que el ancho del entorno.
- Porcentaje de agentes suciedad con respecto al tamaño del entorno.
- Tiempo maximo establecido para la ejecución sistema.


## Proceso de simulación
1. Se establecen los valores para las variables iniciales del modelo.
2. Se ejecuta el modelo.
3. Termina la ejecucion del modelo ya sea porque llego al maximo o porque ya no hay celdas que limpiar.
4. Despliega los resultados.
5. Muestra la simulacion de manera gráfica.


## Resultados
Los resultados que se obtuvieron del modelo a simular fueron exitosos. El sistema cumple satisfactoriamente con las condiciones establecidas, ya que es capaz de crear un entorno de MxN dimensiones, posicionar a los agentes en este, y terminar su ejecución ya sea por exceder su tiempo maximo extablecido o porque ya no hay mas celdas que limpiar. La simulacion gráfica del modelo tambien es la esperada, ya que tiene el comportamiento que desde un principio de esperaba.

In [4]:
import mesa
import numpy as np
import time

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mlp
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
mlp.rcParams['animation.embed_limit'] = 2**128

old_cmap = mlp.cm.get_cmap('viridis', 3)
colors = old_cmap(np.linspace(0, 1, 3))

colors[0] = np.array([10/256, 10/256, 10/256, 1]) # Black
colors[1] = np.array([145/256, 94/256, 55/256, 1]) # Brown
colors[2] = np.array([250/256, 250/256, 250/256, 1]) # White

colormap = mlp.colors.ListedColormap(colors)


# Datos iniciales para la simulacion del modelo
# NOTA: Estos datos pueden ser modificados por el usuario
#       y serviran para probar el modelo con distintos parametros
ROOM_WIDTH = 10
ROOM_HEIGHT = 10
DIRT_PERCENTAGE = 0.1 # Porcentaje en notacion decimal (e.g., 0.1 == 10%)
MAX_EXEC_TIME = 0.002 # Tiempo en segundos
NUM_GENERATIONS = 100


# Clase para el agente 'Robot'
class RobotAgent(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
    
    def step(self):
        self.move()

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )

        position_to_move = self.random.choice(possible_steps)

        if self.model.grid.is_cell_empty(position_to_move):
            self.model.grid.move_agent(self, position_to_move)
            self.model.robots_total_moves += 1
        else:
            agent = self.model.grid.get_cell_list_contents(position_to_move)[0]
            if isinstance(agent, DirtAgent):
                self.model.grid.remove_agent(agent)
                self.model.dirty_cells_quantity -= 1
                self.model.grid.move_agent(self, position_to_move)


# Clase para el agente 'Dirt'
class DirtAgent(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)


# Clase del modelo donde se ejecutaran los agentes
class RoomCleaningModel(mesa.Model):
    def __init__(self, width, height, dirt_percentage):
        self.width = width
        self.height = height
        self.dirt_percentage = dirt_percentage
        self.dirty_cells_quantity = int((self.width * self.height) * self.dirt_percentage)
        self.robots_total_moves = 0
        self.reset()


    def reset(self):
        # Inicializar modo de grilla y modo de activacion de agentes
        self.grid = mesa.space.SingleGrid(self.width, self.height, True)
        self.schedule = mesa.time.SimultaneousActivation(self)

        # Inicializar las celdas sucias y los robots
        self.initialize_dirty_cells()
        self.initialize_robots()

        # Inicializamos el recolector de datos
        self.datacollector = mesa.datacollection.DataCollector(
            model_reporters={"Grid": self.get_grid})

        self.datacollector.collect(self)


    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)


    def initialize_dirty_cells(self):
        # dirty_cells_quantity = int((self.width * self.height) * self.dirt_percentage)

        # Funcion auxiliar para obtener las coordenadas aleatorias para
        # los agentes de suciedad
        def generate_coords():
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            if x == 0: x += 1
            
            return (x, y)

        for _ in range(self.dirty_cells_quantity):
            # Obtener valores de posicion aleatorios
            x, y = generate_coords()

            # En caso de que se generen valores de posiciones aleatorias que resultan
            # coincidir con valores de posiciones de celdas ya ocupadas, se deberan
            # generar nuevas posiciones hasta encontrar una en la que su celda este
            # disponible
            while not self.grid.is_cell_empty((x, y)):
                x, y = generate_coords()

            # Crear agente y posicionarlo en la grilla
            dirt_agent = DirtAgent((x, y), self)
            self.schedule.add(dirt_agent)
            self.grid.place_agent(dirt_agent, (x, y))


    def initialize_robots(self):
        robots_quantity = self.width

        for y in range(robots_quantity):
            robot_agent = RobotAgent(y, self)
            self.schedule.add(robot_agent)
            self.grid.place_agent(robot_agent, (0, y))


    def get_grid(self):
        grid = np.zeros((self.grid.width, self.grid.height))

        for (agent, x, y) in self.grid.coord_iter():
            if isinstance(agent, RobotAgent): grid[x][y] = 0
            elif isinstance(agent, DirtAgent): grid[x][y] = 1
            else: grid[x][y] = 2

        return grid

    
    def getDirtyCellsPercentage(self):
        return (self.dirty_cells_quantity * 100) / (self.width * self.height)


model = RoomCleaningModel(ROOM_WIDTH, ROOM_HEIGHT, DIRT_PERCENTAGE)

start_time = time.time()
final_time = None

for i in range(NUM_GENERATIONS):
    curr_time = time.time() - start_time
    if curr_time >= MAX_EXEC_TIME:
        final_time = MAX_EXEC_TIME
        NUM_GENERATIONS = i
        break
    if model.getDirtyCellsPercentage() == 0.0:
        final_time = curr_time
    model.step()

if final_time == None: final_time = time.time()

print(f'Tiempo maximo establecido: {MAX_EXEC_TIME}s')
print(f'Tiempo de ejecucion: {final_time}s')
print(f'Porcentaje de celdas sucias: {model.getDirtyCellsPercentage()}%')
print(f'Porcentaje de celdas limpias: {100 - model.getDirtyCellsPercentage()}%')
print(f'Numero de movimientos realizados por todos los agentes: {model.robots_total_moves}')
    
all_grids = model.datacollector.get_model_vars_dataframe()
all_grids

Tiempo maximo establecido: 0.002s
Tiempo de ejecucion: 0.002s
Porcentaje de celdas sucias: 4.0%
Porcentaje de celdas limpias: 96.0%
Numero de movimientos realizados por todos los agentes: 98


,Grid
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,"[[2.0, 2.0, 0.0, 2.0, 0.0, 2.0, 0.0, 2.0, 0.0,..."
2,"[[2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 0.0, 2.0,..."
3,"[[2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 2.0, 2.0, 2.0,..."
4,"[[0.0, 2.0, 2.0, 2.0, 0.0, 2.0, 0.0, 0.0, 2.0,..."
5,"[[2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 2.0, 2.0, 2.0,..."
6,"[[2.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 2.0, 2.0,..."
7,"[[2.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0,..."
8,"[[2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0,..."
9,"[[2.0, 0.0, 2.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0,..."


In [5]:
%%capture

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(7,7))

axs.set_xticks([])
axs.set_yticks([])

patch_grid = axs.imshow(model.get_grid(), cmap=colormap)

def animate(i):
    patch_grid.set_data(all_grids['Grid'].iloc[i])

anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS+1)

In [6]:
anim